In [61]:
import os
import json
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

from lab_v2.io import read_file


In [62]:
datasets = ['draw', 'csqa', 'last_letters']
temps = ['0.3', '0.5', '0.7']

ATTRIBUTES = ["majority_distance", "majority_distance_squared", "shannon_entropy", "gini_impurity"]
CLASS = 'majority_correct'

results = []


In [63]:
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(5, activation='relu'),
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dense(25, activation='relu'),
        tf.keras.layers.Dense(100, activation='relu'),
        tf.keras.layers.Dense(1, activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    return model

In [64]:
# Iterate through datasets and temperature levels
for dataset in datasets:
    for temp in temps:
        file_path = f'data/{dataset}/{dataset}-T{temp}.jsonl'
        data = read_file(file_path)
        
        X = data[ATTRIBUTES]
        y = data[CLASS]

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        train_dataset = tf.data.Dataset.from_tensor_slices((X_train.values, y_train.values))
        test_dataset = tf.data.Dataset.from_tensor_slices((X_test.values, y_test.values))

        # Shuffle and batch the data
        train_dataset = train_dataset.shuffle(len(X_train)).batch(1)
        test_dataset = test_dataset.batch(1)

        model = create_model()

        model.fit(train_dataset, epochs=20, verbose=0)

        _, accuracy = model.evaluate(test_dataset, verbose=0)

        results.append({"dataset": dataset, "temperature": temp, "accuracy": accuracy})

results_df = pd.DataFrame(results)

c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\backend.py:5805: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\backend.py:5805: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\backend.py:5805: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
c:\Users\User\AppData\Local\Programs\Python

        dataset temperature  accuracy
0          draw         0.3  0.845000
1          draw         0.5  0.830846
2          draw         0.7  0.820895
3          csqa         0.3  0.612109
4          csqa         0.5  0.608004
5          csqa         0.7  0.571575
6  last_letters         0.3  0.203333
7  last_letters         0.5  0.363333
8  last_letters         0.7  0.573333


In [67]:
results_df

,dataset,temperature,accuracy
0,draw,0.3,0.845000
1,draw,0.5,0.830846
2,draw,0.7,0.820895
3,csqa,0.3,0.612109
4,csqa,0.5,0.608004
5,csqa,0.7,0.571575
6,last_letters,0.3,0.203333
7,last_letters,0.5,0.363333
8,last_letters,0.7,0.573333
